<a href="https://colab.research.google.com/github/as3asddd/MLFinalProject/blob/main/Fine-Pruning%20Approach%20On%20Repairing%20BadNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Pruning Approach On Repairing BadNet

Co-Worker:  
Tieliu Zhou, tz1430, N14973953;  
Juntao Jiang, jj2910, N10249972;  
Kangning Li, kl3596, N12602103;  
Jiazhen Han, jh6419, 


## Data Preparation



This code is to download the data,  
if the execution fails because of permission,  
please execute the fourth code and put the data in the matching position.

In [ ]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

!gdown https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
!gdown https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
!gdown https://drive.google.com/uc?id=1XFKaTse6gflUFK7lDPxXBUaq4oQA8-qy
!gdown https://drive.google.com/uc?id=145qetPRDNRfvNXjjHhHFwPZ8CMfBUGOp

!mkdir data
!mkdir data/multi-trigger
!mv /content/*.h5 /content/data

!gdown https://drive.google.com/uc?id=1SrObV38DPLgsMfpPYTdeX7nzjrEUAEwW
!gdown https://drive.google.com/uc?id=1TiBviHoi-nh-aDRCP-1ZQlP0Nis6wOCw
!gdown https://drive.google.com/uc?id=1TiBviHoi-nh-aDRCP-1ZQlP0Nis6wOCw
!mv /content/*.h5 /content/data/multi-trigger

Cloning into 'CSAW-HackML-2020'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 164 (delta 11), reused 18 (delta 6), pack-reused 141
Receiving objects: 100% (164/164), 79.56 MiB | 37.23 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Permission denied: https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
Maybe you need to change permission over 'Anyone with the link'?
Permission denied: https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
Maybe you need to change permission over 'Anyone with the link'?
Permission denied: https://drive.google.com/uc?id=1XFKaTse6gflUFK7lDPxXBUaq4oQA8-qy
Maybe you need to change permission over 'Anyone with the link'?
Permission denied: https://drive.google.com/uc?id=145qetPRDNRfvNXjjHhHFwPZ8CMfBUGOp
Maybe you need to change permission over 'Anyone with the link'?
mv: cannot stat '/content/*.h5': No such file or directory
Permi

In [ ]:
import os
import sys
import gdown
import tempfile
import copy
import keras
import keras.backend as K
from keras import metrics
import h5py
import numpy as np
import tensorflow as tf


K.clear_session()

if not tf.config.experimental.list_physical_devices('GPU'):
    raise Exception("Please change runtime type to GPU")
!pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

In [ ]:
def data_preprocess(x_data):
    return x_data/255

def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return data_preprocess(x_data), y_data

The code below is also for downloading data,  
so if the first code is executed,  
there is no need to execute this one.

In [ ]:
path = "/content/"

os.chdir(path)
os.listdir(path)
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

from google.colab import drive
drive.mount('/content/drive/')

path = "/content/drive/My Drive/ML For Cyber"
os.chdir(path)
os.listdir(path)

fatal: destination path 'CSAW-HackML-2020' already exists and is not an empty directory.
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


['Assignment1',
 'CSAW-HackML-2020-master',
 'face',
 'data',
 'CSAW-HackML-2020',
 'RepairedNet']

### Load Data

In [ ]:
clean_validation_data_path = "data/clean_validation_data.h5"
clean_validation_data, clean_v_y = data_loader(clean_validation_data_path)

clean_test_data_path = "data/clean_test_data.h5"
clean_test_data, clean_t_y = data_loader(clean_test_data_path)

sunglass_poisoned_test_data_path = "data/sunglasses_poisoned_data.h5"
sunglass_poisoned_test_data, sun_p_t_y = data_loader(sunglass_poisoned_test_data_path)

anonymous_poisoned_data_path = "data/anonymous_1_poisoned_data.h5"
anonymous_poisoned_data, ano_p_y = data_loader(anonymous_poisoned_data_path)

For Multi-trigger Multi-target

In [ ]:
eyebrows_poisoned_data_path = "data/multi-trigger/eyebrows_poisoned_data.h5"
eyebrows_poisoned_data, eye_p_y = data_loader(eyebrows_poisoned_data_path)

lipstick_poisoned_data_path = "data/multi-trigger/lipstick_poisoned_data.h5"
lipstick_poisoned_data, lip_p_y = data_loader(lipstick_poisoned_data_path)

sunglass_poisoned_data_path = "data/multi-trigger/sunglasses_poisoned_data.h5"
sunglass_poisoned_data, sun_p_y = data_loader(sunglass_poisoned_data_path)

In [ ]:
path = "/content/"

os.chdir(path)
os.listdir(path)

['.config', 'drive', 'CSAW-HackML-2020', 'sample_data']

### Data Transformation

In [ ]:
print(np.min(clean_t_y))
print(np.max(clean_t_y))

0
1282


The labels' range is 0 to 1282. (0~N, N=1282, total=1283)

In [ ]:
c_t_x = clean_test_data
c_t_y = keras.utils.to_categorical(clean_t_y, np.max(clean_t_y).astype(np.int32) + 1)
print(c_t_x.shape)
print(c_t_y.shape)

c_v_x = clean_validation_data
c_v_y = keras.utils.to_categorical(clean_v_y, np.max(clean_v_y).astype(np.int32) + 1)
print(c_v_x.shape)
print(c_v_y.shape)

s_p_x = sunglass_poisoned_test_data
s_p_y = keras.utils.to_categorical(sun_p_t_y, 1283)
print(s_p_x.shape)
print(s_p_y.shape)

a_p_x = anonymous_poisoned_data
a_p_y = keras.utils.to_categorical(ano_p_y, 1283)
print(a_p_x.shape)
print(a_p_y.shape)

(12830, 55, 47, 3)
(12830, 1283)
(11547, 55, 47, 3)
(11547, 1283)
(12830, 55, 47, 3)
(12830, 1283)
(10264, 55, 47, 3)
(10264, 1283)


## Load Models

In [ ]:
B1_path = "CSAW-HackML-2020/models/sunglasses_bd_net.h5"
B1_weights = "CSAW-HackML-2020/models/sunglasses_bd_weights.h5"
B1 = keras.models.load_model(B1_path)
B1.load_weights(B1_weights)
#B1.summary()

B2_path = "CSAW-HackML-2020/models/anonymous_1_bd_net.h5"
B2_weights = "CSAW-HackML-2020/models/anonymous_1_bd_weights.h5"
B2 = keras.models.load_model(B2_path)
B2.load_weights(B2_weights)
#B2.summary()

B3_path = "CSAW-HackML-2020/models/anonymous_2_bd_net.h5"
B3_weights = "CSAW-HackML-2020/models/anonymous_2_bd_weights.h5"
B3 = keras.models.load_model(B3_path)
B3.load_weights(B3_weights)
#B3.summary()

BN_path = "CSAW-HackML-2020/models/multi_trigger_multi_target_bd_net.h5"
BN_weights = "CSAW-HackML-2020/models/multi_trigger_multi_target_bd_weights.h5"
BN = keras.models.load_model(BN_path)
BN.load_weights(BN_weights)
#B4.summary()

In [ ]:
B = [B1,B2,B3,BN]

### Evaluate the original models

In [ ]:
def validation(y, y_hat):
    n = 0
    for i, v in enumerate(y):
        correct = 1 if np.argmax(v) == np.argmax(y_hat[i]) else 0
        n+= correct 
    return n/(i + 1)

def eval_on_test(B):
  for i, b in enumerate(B):
    K.clear_session()
    c_y_hat = b.predict(c_t_x)
    bi = "N" if (i+1)==4 else "{t}".format(t=(i+1))
    print("B{bi}".format(bi = bi),"on clean_test_data accuracy: ",validation(c_t_y, c_y_hat)) 
    s_p_y_hat = b.predict(s_p_x)
    print("B{bi}".format(bi = bi),"on sunglasses_poisoned_data accuracy: ",validation(s_p_y, s_p_y_hat))
    a_p_y_hat = b.predict(a_p_x)
    print("B{bi}".format(bi = bi),"on anonymous_1_poisoned_data: ",validation(a_p_y, a_p_y_hat))
    print("\n")

def my_eval(models, category, name):
  for i,m in enumerate(models):
    mi = "N" if (i+1)==4 else "{t}".format(t=(i+1))
    print("{str} {T}{mi}".format(str=category, T=name, mi=mi),"on clean_test_data:")
    m.evaluate(c_t_x, c_t_y, verbose=2)
    print("{str} {T}{mi}".format(str=category, T=name, mi=mi),"on sunglasses_poisoned_data:")
    m.evaluate(s_p_x, s_p_y, verbose=2)
    print("{str} {T}{mi}".format(str=category, T=name, mi=mi),"on anonymous_1_poisoned_data:")
    m.evaluate(a_p_x, a_p_y, verbose=2)
    print("\n"); 

In [ ]:
eval_on_test(B)

B1 on clean_test_data accuracy:  0.9777864380358535
B1 on sunglasses_poisoned_data accuracy:  0.9999220576773188
B1 on anonymous_1_poisoned_data:  0.0010717069368667187


B2 on clean_test_data accuracy:  0.971862821512081
B2 on sunglasses_poisoned_data accuracy:  0.0
B2 on anonymous_1_poisoned_data:  0.913971161340608


B3 on clean_test_data accuracy:  0.9596258768511302
B3 on sunglasses_poisoned_data accuracy:  0.0
B3 on anonymous_1_poisoned_data:  0.0


BN on clean_test_data accuracy:  0.9600935307872175
BN on sunglasses_poisoned_data accuracy:  0.0
BN on anonymous_1_poisoned_data:  0.0




## Fine-Pruning

### Prune

Code Referrence:  
https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras  
https://blog.csdn.net/leo_whj/article/details/108624915

In [ ]:
# train = clean_validation
def Prune(B, train_data, train_labels): 
  pruned_models = []
  for i, b in enumerate(B):
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    # Compute end step to finish pruning after 2 epochs.
    # batch_size = 128
    epochs = 3
    validation_split = 0.1 # 10% of training set will be used for validation set.
    num_data = train_data.shape[0] * (1 - validation_split)
    end_step = np.ceil(num_data).astype(np.int32) * epochs 

    # Define model for pruning.
    pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
    }

    model_for_pruning = prune_low_magnitude(b, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=metrics.categorical_accuracy)
    
    #Train and evaluate the model against baseline
    logdir = tempfile.mkdtemp()

    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
      tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]
    model_for_pruning.fit(train_data, train_labels,
                  epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

    pruned_models.append(model_for_pruning)

  return pruned_models  

In [ ]:
pruned_models = Prune(B, c_v_x, c_v_y)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/3
325/325 [==============================] - 16s 19ms/step - loss: 0.8060 - categorical_accuracy: 0.8473 - val_loss: 0.5359 - val_categorical_accuracy: 0.8918
Epoch 2/3
325/325 [==============================] - 5s 16ms/step - loss: 0.1782 - categorical_accuracy: 0.9634 - val_loss: 0.5563 - val_categorical_accuracy: 0.8952
Epoch 3/3
325/325 [==============================] - 5s 16ms/step - loss: 0.0996 - categorical_accuracy: 0.9726 - val_loss: 0.7092 - val_categorical_accuracy: 0.8823
Epoch 1/3
325/325 [==============================] - 9s 18ms/step - loss: 0.4987 - categorical_accuracy: 0.8995 - val_loss: 0.3253 - val_categorical_accuracy: 0.9385
Epoch 2/3
325/325 [==============================] - 5s 17ms/step - loss: 0.1063 - categorical_accuracy: 0.9762 - val_loss: 0.3457 - val_categorical_accuracy: 0.9255
Epoch 3/3
325/325 [==============================] - 5s 17ms/step - loss: 0.0566 - categorical_accuracy: 0.9870 - val_loss: 0.4502 - val_categorical_accuracy: 0.9082
Epo

### Fine-tune

In [ ]:
def Fine_tune(pruned_models, train_data, train_labels):
  repaired_models = []
  for i, pm in enumerate(pruned_models):
    G = copy.copy(pm)
    epochs = 5
    validation_split = 0.1

    G.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=metrics.categorical_accuracy)

    G.fit(train_data, train_labels,
          epochs=epochs, 
          validation_split=validation_split)
    
    repaired_models.append(G)

  return repaired_models


In [ ]:
G = Fine_tune(pruned_models, c_v_x, c_v_y)

Epoch 1/5
325/325 [==============================] - 7s 14ms/step - loss: 0.2113 - categorical_accuracy: 0.9406 - val_loss: 0.8144 - val_categorical_accuracy: 0.8736
Epoch 2/5
325/325 [==============================] - 4s 13ms/step - loss: 0.0995 - categorical_accuracy: 0.9713 - val_loss: 0.8909 - val_categorical_accuracy: 0.8745
Epoch 3/5
325/325 [==============================] - 4s 13ms/step - loss: 0.0892 - categorical_accuracy: 0.9721 - val_loss: 1.0219 - val_categorical_accuracy: 0.8554
Epoch 4/5
325/325 [==============================] - 4s 13ms/step - loss: 0.0862 - categorical_accuracy: 0.9774 - val_loss: 0.9636 - val_categorical_accuracy: 0.8537
Epoch 5/5
325/325 [==============================] - 4s 13ms/step - loss: 0.0862 - categorical_accuracy: 0.9771 - val_loss: 1.0579 - val_categorical_accuracy: 0.8519
Epoch 1/5
325/325 [==============================] - 7s 14ms/step - loss: 0.1050 - categorical_accuracy: 0.9684 - val_loss: 0.4712 - val_categorical_accuracy: 0.9082
Epoc

### Evaluate Repaired BadNet

In [ ]:
my_eval(G, "Repaired","G")

Repaired G1 on clean_test_data:
401/401 - 1s - loss: 0.9278 - categorical_accuracy: 0.8683
Repaired G1 on sunglasses_poisoned_data:
401/401 - 1s - loss: 13.5371 - categorical_accuracy: 0.0811
Repaired G1 on anonymous_1_poisoned_data:
321/321 - 1s - loss: 35.1173 - categorical_accuracy: 0.0012


Repaired G2 on clean_test_data:
401/401 - 1s - loss: 0.6357 - categorical_accuracy: 0.8985
Repaired G2 on sunglasses_poisoned_data:
401/401 - 1s - loss: 40.6335 - categorical_accuracy: 0.0000e+00
Repaired G2 on anonymous_1_poisoned_data:
321/321 - 1s - loss: 10.2133 - categorical_accuracy: 0.0843


Repaired G3 on clean_test_data:
401/401 - 1s - loss: 0.5639 - categorical_accuracy: 0.9065
Repaired G3 on sunglasses_poisoned_data:
401/401 - 1s - loss: 72.8628 - categorical_accuracy: 0.0000e+00
Repaired G3 on anonymous_1_poisoned_data:
321/321 - 1s - loss: 45.2126 - categorical_accuracy: 9.7428e-05


Repaired GN on clean_test_data:
401/401 - 1s - loss: 0.5886 - categorical_accuracy: 0.9067
Repaired 

The accuracy is so low that I think there may be adaptive attck  
that cannot be solved by simply prune the unactive neurons.  
Another assumption is that  
maybe the original sunglass net B1 is a good one, as well as anymous_1 B2,  
according to the comparation of the result before repairing and after repairing.  
Therefore, we should use those two data and nets to train and Fine-pruning.

### Save Repaired Net

In [ ]:
def Save_G(G):
  for i, g in enumerate(G):
    gi = "N" if (i+1)==4 else "{t}".format(t=(i+1))
    g.save("/content/drive/My Drive/ML For Cyber/RepairedNet/G{gi}.h5".format(gi=gi))
    g.save_weights("/content/drive/My Drive/ML For Cyber/RepairedNet/G{gi}_weights.h5".format(gi=gi))

Save_G(G)

## Backdoored data detection

We classify those different results of BadNets B and RepairedNets G as poisoned inputs  
that we put them in the N+1 class. (original labels are 0 to N, N=1282, N+1=1283)  
BN, GN and data that under "Multi-trigger Multi-target" are used, as well as clean_validation_data.

In [ ]:
def backdoor_classify(y_hat_bad, y_hat_repair):
  new_y_hat = np.zeros((len(y_hat_repair), 1284))
  for i in range(len(y_hat_repair)): # size of samples
    pos_1 = np.argmax(y_hat_bad[i])
    pos_2 = np.argmax(y_hat_repair[i])

    if(pos_1 == pos_2):
      new_y_hat[i][pos_1] = 1
    else:
      new_y_hat[i][-1] = 1
  return new_y_hat

### Load Model of BN and GN

In [ ]:
K.clear_session()
BN = keras.models.load_model(BN_path)
BN.load_weights(BN_weights)

In [ ]:
GN = G[3]
#GN_path = "/content/drive/My Drive/ML For Cyber/RepairedNet/GN.h5"
#GN_weights = "/content/drive/My Drive/ML For Cyber/RepairedNet/GN_weights.h5"
#GN = keras.models.load_model(GN_path)

### Data Process

In [ ]:
n = 1283
e_p_x = eyebrows_poisoned_data
e_p_y = keras.utils.to_categorical(eye_p_y, n)
print(e_p_x.shape)
print(e_p_y.shape)

l_p_x = lipstick_poisoned_data
l_p_y = keras.utils.to_categorical(lip_p_y, n)
print(l_p_x.shape)
print(l_p_y.shape)

s_p_x2 = sunglass_poisoned_data
s_p_y2 = keras.utils.to_categorical(sun_p_y, n)
print(s_p_x2.shape)
print(s_p_y2.shape)

(10264, 55, 47, 3)
(10264, 1283)
(10264, 55, 47, 3)
(10264, 1283)
(10264, 55, 47, 3)
(10264, 1283)


In [ ]:
c_y_hat_bad = BN.predict(c_v_x)
c_y_hat_repair = GN.predict(c_v_x)

e_y_hat_bad = BN.predict(e_p_x)
l_y_hat_bad = BN.predict(l_p_x)
s_y_hat_bad = BN.predict(s_p_x2)

e_y_hat_repair = GN.predict(e_p_x)
l_y_hat_repair = GN.predict(l_p_x)
s_y_hat_repair = GN.predict(s_p_x2)

e_new_y_hat = backdoor_classify(e_y_hat_bad, e_y_hat_repair)
l_new_y_hat = backdoor_classify(l_y_hat_bad, l_y_hat_repair)
s_new_y_hat = backdoor_classify(s_y_hat_bad, s_y_hat_repair)

c_new_y_hat = backdoor_classify(c_y_hat_bad, c_y_hat_repair)

In [ ]:
print(c_new_y_hat.shape)

(11547, 1284)


### Evaluate

In [ ]:
def N_puls_one_validation(y, y_hat):
  samples = y.shape[0]
  count = 0
  for i in range(samples):
    if np.argmax(y[i]) == np.argmax(y_hat[i]):
      count+=1

  acc = float(count)/samples
  print("Accuracy: {ans}\n".format(ans=acc))
  print("\n")

In [ ]:
print("Use BN and GN:\n")
print("For Clean Validation Data:\n")
N_puls_one_validation(c_v_y, c_new_y_hat)

print("For Eyebrows Poisoned Data Under Multi-trigger:\n")
N_puls_one_validation(e_p_y, e_new_y_hat)

print("For Lipstick Poisoned Data Under Multi-trigger:\n")
N_puls_one_validation(l_p_y, l_new_y_hat)

print("For Sunglass Poisoned Data Under Multi-trigger:\n")
N_puls_one_validation(s_p_y2, s_new_y_hat)

Use BN and GN:

For Clean Validation Data:

Accuracy: 0.949077682514939



For Eyebrows Poisoned Data Under Multi-trigger:

Accuracy: 9.742790335151988e-05



For Lipstick Poisoned Data Under Multi-trigger:

Accuracy: 0.46463367108339826



For Sunglass Poisoned Data Under Multi-trigger:

Accuracy: 0.23265783320342948



